### pyAPES MLM benchmarking at US-Prr

Demo how to:
1. Retrieve model results using xarray
1. Explore some results and benchmark against measured FluxNet data

We use Poker Flat Research Range Black Spruce FluxNet site (US-Prr) as example

In [1]:
# setting path
import sys
#sys.path.append('c:\\Repositories\\pyAPES_main')
import os
from dotenv import load_dotenv

load_dotenv()
pyAPES_main_folder = os.getenv('pyAPES_main_folder')

sys.path.append(pyAPES_main_folder)
#print(sys.path)

### Import modules

In [2]:
# function to read forcing data. See 'forcing/forcing_info.txt' for model forcing variable names and units!
from pyAPES.utils.iotools import read_forcing

# python packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

eps = 1e-16

### Read model results using xarray
- use function *pyAPES.utils.utils.read_results* to open NetCDF4 file storing the results. It uses xarray, see https://docs.xarray.dev/en/stable/

- compare some model results against fluxes and state variables from US-PRR (FluxNet2015 data subset compiled in *Ex1_creating_model_forcing.ipynb*)

In [3]:
from pyAPES.utils.iotools import read_results, read_data

resultfile = r'results\USPrr_test4.nc'
datafile = r'forcing\US-Prr\US-Prr_FLUXNET2015_SUBSET_2011_2015.dat'

# read simulation restuls to xarray dataset
results = read_results(resultfile)

# read observations from US-Prr
data = read_data(os.path.join(pyAPES_main_folder, datafile), start_time='2011-05-01', end_time='2011-10-31')

In [4]:
print(data.columns)

Index(['year', 'month', 'day', 'hour', 'minute', 'doy', 'NETRAD', 'SW_IN_F',
       'PPFD_IN', 'PPFD_OUT', 'SW_OUT', 'LW_IN_F', 'LW_OUT', 'SW_ALB',
       'PPDF_ALB', 'H_F_MDS', 'H_RANDUNC', 'H_CORR', 'H_CORR_25', 'H_CORR_75',
       'LE_F_MDS', 'LE_RANDUNC', 'LE_CORR', 'LE_CORR_25', 'LE_CORR_75',
       'G_F_MDS', 'NEE_VUT_REF', 'NEE_VUT_REF_RANDUNC', 'GPP_NT_VUT_REF',
       'GPP_DT_VUT_REF', 'RECO_NT_VUT_REF', 'RECO_DT_VUT_REF', 'TA_F',
       'TS_F_MDS_1', 'P_F', 'SWC_F_MDS_2', 'WS_F', 'WD', 'NIGHT', 'SW_IN_F_QC',
       'LW_IN_F_QC', 'H_F_MDS_QC', 'LE_F_MDS_QC', 'G_F_MDS_QC',
       'NEE_VUT_REF_QC', 'VPD_F_QC', 'TA_F_QC', 'P_F_QC'],
      dtype='object')


### Print resuts metadata:

Output variables are those defined at pyAPES.parameters.mlm_outputs'. Each variable contains one or several of following dimensions:

- date --> time dimension
- simulation --> simulation number. If there is only one, remember that it has index 0!
- canopy --> canopy and air layers. Index is 0 at ground and increases upwards
- planttype --> planttypes defined in mlm_parameters. Index follows the order in dict cpara['planttypes']) 
- soil -> soil layers. Index 0 is at the top and increases downwards (nr soil layers, 0 at top)
- groundtype --> groundtypes at forestfloor. Defined in mlm_parameters. Index follows the order in dict: cpara['forestfloor']['bottom_layer_types']


In [5]:
print(results)

# print list of all variables:
vars = list(results.data_vars)
for v in vars:
    print(v)

<xarray.Dataset>
Dimensions:                               (date: 673, simulation: 1,
                                           canopy: 50, planttype: 3, soil: 60,
                                           groundtype: 3)
Coordinates:
  * date                                  (date) datetime64[ns] 2011-06-01 .....
  * simulation                            (simulation) int64 0
  * soil                                  (soil) float32 -0.005 ... -1.975
  * canopy                                (canopy) float32 0.0 0.2245 ... 11.0
Dimensions without coordinates: planttype, groundtype
Data variables: (12/157)
    forcing_air_temperature               (date, simulation) float32 ...
    forcing_precipitation                 (date, simulation) float32 ...
    forcing_pressure                      (date, simulation) float32 ...
    forcing_h2o                           (date, simulation) float32 ...
    forcing_co2                           (date, simulation) float32 ...
    forcing_wind_speed

### Plot some model results and compare with observations at US-Prr

- use FluxNet2015 data subset created in *Ex1_creating_model_forcing.ipynb*

In [6]:
sim = 0  # in this demo, we have only one simulation (i.e. only one parameter set was used)

# grid variables for plotting
t = results.date  # time
zc = results.canopy_z  # height above ground [m]
zs = results.soil_z  # depth of soil; shown negative [m]

### Soil temperature and moisture
- computed using *pyAPES.soil* submodels 'Water' and 'Heat'
- compare with measurements at similar depths (to be done - now compare with Ts and SWC at 5cm depth)

In [7]:
%matplotlib qt
var = ['soil_temperature', 'soil_volumetric_liquid_water_content', 'soil_volumetric_ice_content']

lyrs = [0, 6, 25, 35] # layers
#depths = np.array2string(np.asarray(zs[lyrs]), precision=1, separator=', ')
depths = ['{:.2f} m'.format(k) for k in zs[lyrs]]

fig, ax = plt.subplots(len(var), 1, figsize=(10,7))

k = 0
ax[0].plot(t, data['TS_F_MDS_1'], 'ko', markersize=3, alpha=0.1)
ax[1].plot(t, 1e-2 * data['SWC_F_MDS_2'], 'ko', markersize=3, alpha=0.1)
for v in var:
    ax[k].plot(t, results[v][:,sim,lyrs], label=depths)
    ax[k].set_ylabel(results[v].attrs['units'])
    ax[k].tick_params(axis='x', labelrotation = 20)
    ax[k].legend(fontsize=8)
    k += 1

# # vertical profile at last timestep
# fig, ax = plt.subplots(1, 2) #figsize=(10,5))

# k = 0
# for v in var:
#     ax[k].plot(results[v][-1,sim,:], zs)
#     ax[k].set_xlabel(results[v].attrs['units'])
#     ax[0].set_ylabel('depth (m)')
#     k += 1

ValueError: x and y must have same first dimension, but have shapes (673,) and (8785,)

### Ecosystem-scale fluxes

- ecosystem - atm. fluxes represent the integrated sinks / sources in soil (soil-module), forestfloor (bottomlayer-module) and vegetation (planttype&canopy -modules)
- comparable to ecosystem - atmosphere exchange
- affected by current model forcing and sub-model instance state (e.g. Planttype and Canopy LAI, phenology, soil temperature, moisture etc.)


In [8]:
%matplotlib qt
# var = ['forcing_air_temperature', 'forcing_par', 'canopy_NEE', 'canopy_GPP', 
#        'canopy_Reco', 'canopy_Rnet', 'canopy_SH', 'canopy_LE', 'ffloor_ground_heat']

var = ['canopy_NEE', 'canopy_GPP', 'canopy_Reco']
var2 = ['canopy_Rnet', 'canopy_SH', 'canopy_LE', 'ffloor_ground_heat']


# # temperature & par
# ax[0].plot(t, results[var[0]][:,sim], 'k-', label=var[0])
# ax[0].set_ylabel(results[var[0]].attrs['units'])
# ax[0].tick_params(axis='x', labelrotation = 20)
# axb = ax[0].twinx()

# axb.plot(t, results[var[1]][:,sim], 'r-', label=var[1]) # Par
# axb.set_ylabel(results[var[1]].attrs['units'])

fig, ax = plt.subplots(3, 1, figsize=(8,12), sharex=True)
# CO2 fluxes
ax[0].plot(t, data['NEE_VUT_REF'], 'k.-', alpha=0.3)
ax[1].plot(t, data['GPP_NT_VUT_REF'], 'k.-', alpha=0.3)
ax[2].plot(t, data['RECO_NT_VUT_REF'], 'k.-', alpha=0.3)
n = 0
for v in var:
    ax[n].plot(t, results[v][:,sim], label=v)

    ax[n].set_ylabel('umol m-2 s-1')
    ax[n].tick_params(axis='x', labelrotation = 20)
    ax[n].legend(fontsize=8)
    n +=1

fig, ax = plt.subplots(5, 1, figsize=(8,12), sharex=True)
# energy fluxes
ax[1].plot(t, data['NETRAD'], 'k.-', alpha=0.3)
ax[2].plot(t, data['H_F_MDS'], 'k.-', alpha=0.3)
ax[3].plot(t, data['LE_F_MDS'], 'k.-', alpha=0.3)
ax[4].plot(t, data['G_F_MDS'], 'k.-', alpha=0.3)

n = 1
for v in var2:
    ax[0].plot(t, results[v][:,sim], label=v)
    ax[n].plot(t, results[v][:,sim], label=v)

    ax[n].set_ylabel('W m-2')
    ax[n].tick_params(axis='x', labelrotation = 20)
    ax[n].legend(fontsize=8)
    n += 1


ValueError: x and y must have same first dimension, but have shapes (673,) and (8785,)

In [10]:
fig, ax = plt.subplots(4, 1, figsize=(8,12), sharex=True)
# CO2 fluxes
ax[0].plot(t, data['GPP_NT_VUT_REF'], 'k.-', alpha=0.3); ax[0].set_ylabel('GPP (umolm-2s-1)')
ax[0].plot(t, results['canopy_GPP'][:,sim], label='GPP')

ax[1].plot(t, data['RECO_NT_VUT_REF'], 'k.-', alpha=0.3); ax[1].set_ylabel('RECO (umolm-2s-1)')
ax[1].plot(t, results['canopy_Reco'][:,sim], label='GPP')
ax[2].plot(t, data['TS_F_MDS_1'], 'k.-', alpha=0.3); ax[2].set_ylabel('T (degC)')
ax[3].plot(t, 1e-2*data['SWC_F_MDS_2'], 'k.-', alpha=0.3); ax[3].set_ylabel('SWC (m3m-3)')

ax[2].plot(t, results['soil_temperature'][:,sim,lyrs], label=depths)
ax[2].legend(fontsize=8)

var3 = ['soil_volumetric_liquid_water_content', 'soil_volumetric_ice_content']
k = 3
for v in var3:
    ax[k].plot(t, results[v][:,sim,lyrs], label=depths)
    ax[k].set_ylabel(results[v].attrs['units'])
    ax[k].tick_params(axis='x', labelrotation = 20)
    ax[k].legend(fontsize=8)



In [11]:
print(data.columns)

Index(['year', 'month', 'day', 'hour', 'minute', 'doy', 'NETRAD', 'SW_IN_F',
       'PPFD_IN', 'PPFD_OUT', 'SW_OUT', 'LW_IN_F', 'LW_OUT', 'SW_ALB',
       'PPDF_ALB', 'H_F_MDS', 'H_RANDUNC', 'H_CORR', 'H_CORR_25', 'H_CORR_75',
       'LE_F_MDS', 'LE_RANDUNC', 'LE_CORR', 'LE_CORR_25', 'LE_CORR_75',
       'G_F_MDS', 'NEE_VUT_REF', 'NEE_VUT_REF_RANDUNC', 'GPP_NT_VUT_REF',
       'GPP_DT_VUT_REF', 'RECO_NT_VUT_REF', 'RECO_DT_VUT_REF', 'TA_F',
       'TS_F_MDS_1', 'P_F', 'SWC_F_MDS_2', 'WS_F', 'WD', 'NIGHT', 'SW_IN_F_QC',
       'LW_IN_F_QC', 'H_F_MDS_QC', 'LE_F_MDS_QC', 'G_F_MDS_QC',
       'NEE_VUT_REF_QC', 'VPD_F_QC', 'TA_F_QC', 'P_F_QC'],
      dtype='object')


In [12]:
10**0.45


2.8183829312644537

### Ecosystem radiation balance

- net radiation (Rnet) is the sum of net shortwave (SWnet = incoming - reflected) and net longwave (LWnet = incoming - emitted) radiation at canopy top
- computed via models in pyAPES.microclimate.radiation, called iteratively from pyAPES.canopy.mlm_canopy to account for canopy structure and leaf & forest floor temperature
- ecosystem albedo can be computed from radiation profiles at uppermost grid point

In [13]:
# net radiation components at canopy top
var = ['canopy_Rnet','canopy_SWnet', 'canopy_LWnet']
profs = ['canopy_par_down', 'canopy_par_up', 'canopy_nir_down','canopy_nir_up','canopy_lw_down','canopy_lw_up']

fig, ax = plt.subplots(3, 1, figsize=(8,12), sharex=True)

for v in var:
    ax[0].plot(t, results[v][:, sim], label=v)
ax[0].set_ylabel('W m-2')
ax[0].tick_params(axis='x', labelrotation = 20)
ax[0].legend(fontsize=8)    

# lets plot also partitioning of canopy_SWnet and canopy_LWnet.
# -1 is the index of uppermost gridpoint

for v in ['canopy_par_down', 'canopy_nir_down','canopy_lw_down']: # downward
    ax[1].plot(t, results[v][:,sim,-1], '-', label=v)
for v in ['canopy_par_up', 'canopy_nir_up','canopy_lw_up']: # upward
    ax[1].plot(t, results[v][:,sim,-1], '--', label=v)
ax[1].set_ylabel('W m-2')
ax[1].tick_params(axis='x', labelrotation = 20)
ax[1].legend(fontsize=8)

# canopy albedo
eps = 1e-16

# fraction of par on total SW
f_par = results['canopy_par_down'][:,sim,-1] / (results['canopy_par_down'][:,sim,-1] + results['canopy_nir_down'][:,sim,-1] + eps)

alb_par = results['canopy_par_up'][:,sim,-1] / (results['canopy_par_down'][:,sim,-1] + eps)
alb_nir = results['canopy_nir_up'][:,sim,-1] / (results['canopy_nir_down'][:,sim,-1] + eps)
alb_sw = f_par * alb_par + (1 - f_par) * alb_nir
alb_sw = np.maximum(0, np.minimum(1.0, alb_sw))

ax[2].plot(t, alb_sw, '-', label='SW albedo')
ax[2].plot(t, alb_par, '-', label='Par albedo')
ax[2].plot(t, alb_nir, '-', label='Nir albedo')

ax[2].tick_params(axis='x', labelrotation = 20)
ax[2].legend(fontsize=8)

### Microclimatic gradients within the canopy

- sub-models pyAPES.microclimate.Micromet & pyAPES.radiation.Radiation
- mean wind speed & friction velocity (momentum flux) profiles depend on momentum absorption in the canopy, which is proportional to leaf-area density (*lad*) profile. Canopy *lad* is sum of *lad* of each PlantType instance
- SW absorption depends on *lad* and leaf optical properties
- air-space scalar profiles (*T*, *H2O* & *CO2*) in steady-state with respective sink/source profile using 1st-order closure (K-theory)

Let's compute average profiles for daytime hours 10:00 -- 16:00 under conditions when canopy is dry


In [14]:
#par = results['forcing_par'][:,sim] # use PAR as criteria for night / day
ix = np.where((results.date.dt.hour>=10) & (results.date.dt.hour<=16) & (results['canopy_interception_storage'] <= eps))
print(ix)

rad = ['canopy_par']
mmet = ['canopy_wind_speed', 'canopy_friction_velocity', 'canopy_temperature', 'canopy_co2', 'canopy_h2o']

plt.figure('scalarprofiles')
n = 1
for v in var:
    plt.subplot(2,2,n)
    x = results[v][:, sim, :] - results[v][:, sim, -1] # s - s_ref
    xm = np.mean(x, axis=0)
    xmn = np.mean(x.loc[par < 20, :], axis=0) # night
    xmd = np.mean(x.loc[par > 200, :], axis=0) # day
    plt.plot(x, zcm, 'k', alpha=0.1)
    plt.plot(xmd, zc, 'r', label='day'); plt.plot(xmn, zc, 'b', label='night')
    plt.xlabel(results[v].attrs['units']); plt.ylabel(zc.attrs['units'])
    n += 1

plt.legend()


(array([  20,   21,   22,   23,   24,   25,   26,   27,   28,   29,   30,
         31,   32,   33,   68,   69,   70,   71,   72,   73,   74,   75,
         76,   77,   78,   79,   80,   81,  164,  165,  166,  167,  168,
        169,  170,  171,  172,  173,  174,  175,  176,  177,  260,  261,
        262,  263,  264,  265,  266,  267,  268,  269,  270,  271,  272,
        273,  308,  309,  310,  311,  312,  313,  314,  315,  316,  317,
        318,  319,  320,  321,  356,  357,  358,  359,  360,  361,  362,
        363,  364,  365,  366,  367,  368,  369,  404,  405,  406,  407,
        408,  409,  410,  411,  412,  413,  414,  415,  416,  417,  452,
        453,  454,  455,  456,  457,  458,  459,  460,  461,  462,  463,
        464,  465,  500,  501,  502,  503,  504,  505,  506,  507,  508,
        509,  510,  511,  512,  513,  548,  549,  550,  551,  552,  553,
        554,  555,  556,  557,  558,  559,  560,  561,  596,  597,  598,
        599,  600,  601,  602,  603,  604,  605,  

IndexError: too many indices

In [ ]:
plt.plot(results['gt_water_content'][:,0,:])
print(results['canopy_planttypes'].values)
print(results['ffloor_groundtypes'].values)

plt.close('all')

plt.plot(results['ffloor_soil_respiration'])

In [ ]:
results.close()